Install the required dependencies:

Import the packages you'll need:

In [1]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

c:\Users\NEW USER\anaconda3\envs\genv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
p = PyPDFLoader('Comparative_Analysis_of_Machine_Learning.pdf')
docs = p.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 9.26', 'creator': '', 'creationdate': "D:20231110215223Z00'00'", 'moddate': "D:20231110215223Z00'00'", 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'Comparative_Analysis_of_Machine_Learning.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='Comparative Analysis of Machine Learning Algorithms in Predicting Rate of \nPenetration during Drilling\nOlaosebikan Abidoye Olafadehan*, Ikenna David Ahaotu\nDepartment of Chemical and Petroleum Engineering, University of Lagos, Akoka-Y aba, Lagos 101017, Nigeria\nCitation: Olafadehan OA, Ahaotu  ID, Comparative Analysis of Machine Learning Algorithms in Predicting Rate of Penetration \nduring Drilling. J Petro Chem Eng 2023;1(1): 32-47.\nReceived: 14 October, 2023; Accepted: 31 October, 2023; Published: 07 November, 2023\n*Corresponding author: Olafadehan OA, Department of Chemical and Petroleum Engineering, University of Lagos, Akoka-\nY aba, Lagos 101017 N

### Setup

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [30]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_API")
llm=ChatGroq(groq_api_key=api_key,model="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F21690EF80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F21690FDC0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
token=os.getenv("astra_token")
a_id = os.getenv("astra_id")

In [12]:
cassio.init(token=token, database_id=a_id)

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\NEW USER\anaconda3\envs\genv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NEW USER\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
astra_store = Cassandra(embedding=embeddings,table_name='ik_db',session=None,keyspace=None)

In [19]:
from langchain_text_splitters import CharacterTextSplitter
splitter = CharacterTextSplitter(separator='/n',chunk_size=1000,chunk_overlap=300)

In [21]:
docss= splitter.split_documents(docs)

In [23]:
len(docss)

29

In [25]:
astra_store.add_documents(docss)
vec = VectorStoreIndexWrapper(vectorstore=astra_store)

In [34]:
vec.query('How many machine learning where used in this paper',llm=llm)

'According to the text, the following machine learning techniques were used:\n\n1. Random Forest\n2. Support Vector Machine (SVM)\n3. Linear Regression\n4. K-Nearest Neighbors (KNN)\n5. Stacking Technique (which combines the predictions of multiple models)\n6. Voting Technique (which integrates the predictions of multiple models)\n7. Convolutional Neural Network (CNN)\n\nSo, in total, 7 machine learning techniques were used in this paper.'

In [5]:
h = ['i am david, i am good']
'/n'.join(h)

'i am david, i am good'